## 搭建一个分词工具

### Part 1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [1]:
import numpy as np
import pandas as pd
import xlrd

In [2]:
# TODO: 第一步： 从综合类中文词库.xlsx 中读取所有中文词。
#  hint: 思考一下用什么数据结构来存储这个词典会比较好？ 要考虑我们每次查询一个单词的效率。 
def dataRead(xpath, sheet_index=0):
    work_book = xlrd.open_workbook(filename=xpath)
    work_sheet = work_book.sheet_by_index(sheet_index)
    dic_words = {}
    max_length = 0
    for idx in range(work_sheet.nrows):
        word = work_sheet.row(idx)[0].value.strip()
        temp_length = len(word)
        if temp_length > max_length:
            max_length = temp_length
        dic_words[word] = 0.00001
    return dic-words, max_length

dic_words, max_length = dataRead("综合类中文词库.xlsx")    # 保存词典库中读取的单词

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

for key, value in word_prob.items():
    dic_words[key] = value
print (sum(word_prob.values()))

NameError: name 'work_shet' is not defined

In [2]:
def word_cut_native(input_str):
    # 递归的方式枚举出所有的可能
    length_str= len(input_str)
    
    segments = []
    if length_str == 0:
        return segments
    
    max_split = min(max_length, length_str) + 1
    for ids in range(1, max_split):
        word = input_str[0:ids]
        
        if word in dic_words:
            segment_words = word_cut_native(input_str[ids:])
            if len(segment_words) == 0:
                segments.append([word])
            else:
                for element in segment_words:
                    element =[word]+element
                    segments.append(element)
    return segments
    


## TODO 请编写word_segment_naive函数来实现对输入字符串的分词
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """

    length_input_str = len(input_str)
    # TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
    segments = word_cut_native(input_str) # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
                   # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]
    
    # TODO: 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
    best_segment = []
    best_score = np.inf
    for seg in segments:
        # TODO ...
        log_sum= -1 * np.sum(np.log([dic_words[word]+EPSILON  for word in seg]))
        if log_sum <best_score:
            best_score=log_sum
            best_segment=seg
    
    return best_segment      

SyntaxError: invalid syntax (<ipython-input-2-04c3acee1582>, line 18)

In [3]:
# 测试
print (word_segment_naive("北京的天气真好啊"))
print (word_segment_naive("今天的课程内容很有意思")
print (word_segment_naive("经常有意见分歧"))

SyntaxError: invalid syntax (<ipython-input-3-830f0e9adef8>, line 4)

### Part 2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1的要求一致

In [4]:
def build_graph(input_str):
    # 构件图
    chars = list(input_str)
    chars_length = len(chars)
    word_list = [[0 for _ in range(chars_length)] for _ in range(chars_length)]
    
    for i in range(chars_length):
        for j in range(i, chars_length):
            if input_str[i:j+1] in dic_words:
                key = input_str[i:j+1]
                word_list[i][j] = dic_words[key]
                
    graph = pd.DataFrame(data=np.array(word_list), index=list(range(chars_length)))
    graph = -1 * np.log(graph + EPSILON)
    return graph

## TODO 请编写word_segment_viterbi函数来实现对输入字符串的分词
def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    
    # TODO: 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph） 参考：课程内容
    #      有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率在 word_prob，如果不在word_prob里的单词但在
    #      词典里存在的，统一用概率值0.00001。
    #      注意：思考用什么方式来存储这种有向图比较合适？ 不一定有只有一种方式来存储这种结构。 
    graph = build_graph(input_str)
    num_chars = len(input_str)
    distances_min = pd.Series(data=np.zeros(num_chars  + 1), index=list(range(num_char)))
    path =  path=pd.Series( data=np.zeros(shape=(num_chars + 1,),dtype=np.int),index=list(range(num_chars + 1)))
    # TODO： 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    #              hint: 思考为什么不用相乘: p(w1)p(w2)...而是使用negative log sum:  -log(w1)-log(w2)-...
    for idx_end in range(1, num_chars+1):
        possible_path_distances = np.zeros((idx_end, ))
        for idx_start in range(idx_end):
            possible_path_distances[idx_start] = distances_min[idx_start] + graph.loc[idx_start, idx_end]
        distances_min[idx_end] = np.min(possible_path_distances)
    # TODO: 第三步： 根据最好的PATH, 返回最好的切分
    best_segment = []
    idx = num_chars
    while idx > 0:
        best_segment.append(input_str[path.loc[idx]:idx])
        idx = path.loc[idx]
    best_segment.reverse()
    return best_segment  

SyntaxError: invalid syntax (<ipython-input-4-29f9ecd65d37>, line 18)

In [5]:
# 测试
print (word_segment_naive("北京的天气真好啊"))
print (word_segment_naive("今天的课程内容很有意思")
print (word_segment_naive("经常有意见分歧"))

SyntaxError: invalid syntax (<ipython-input-5-830f0e9adef8>, line 4)

In [6]:

# TODO: 第一种方法和第二种方法的时间复杂度和空间复杂度分别是多少？
第一个方法： 
时间复杂度= , 空间复杂度=

第二个方法：
时间复杂度= , 空间复杂度=

SyntaxError: invalid character in identifier (<ipython-input-6-2a298e6f8f52>, line 3)

In [7]:
# TODO：如果把上述的分词工具持续优化，有哪些可以考虑的方法？ （至少列出3点）
- 0. （例）， 目前的概率是不完整的，可以考虑大量的语料库，然后从中计算出每一个词出现的概率，这样更加真实
- 1.
- 2.
- 3. 

SyntaxError: invalid character in identifier (<ipython-input-7-0c7c1ff5f492>, line 2)